## Import library

In [2]:


import os
import pandas as pd
import xml.etree.ElementTree as ET

## Input day text('Thu') and download all the day tragetory files

In [ ]:
# "https://www.flightaware.com/live/findflight?origin=RKSI&destination=RJFF" 진입
# selenium으로 요일 텍스트 'Thu'에 해당하는 링크들 선회
# 클릭해서 새 driver로 전환
# 해당 driver링크 저장 ("https://www.flightaware.com/live/flight/ESR649/history/20251113/0925Z/RKSI/RJFF")
# 위 과정 반복
# 끝나면 url링크 + '/google_earth' 처리 ("https://www.flightaware.com/live/flight/ESR649/history/20251113/0925Z/RKSI/RJFF/google_earth")
# 끝나면 각 링크 순회 -> 파일링크라서 자동 다운로드 됨
# 이후 kaggle dataset에 업로드

## Upload all downloaded files and integrate
> KML : XML 기반, XML은 태그기반의 트리 구조 (HTML처럼)  
> tree → root → child → child … 이런 계층이 존재  
> 다만 화면 표시용이 아닌 데이터를 저장하고 전달하기 위한 언어
> 속성끼리 자연스럽게 묶이고 그룹화되는 구조 표현에 적합
> 계층적 데이터 표현(지도/항공/위치/기업시스템설정), schema를 통한 엄밀한 데이터구조 검증(규격 절대사수: 금융/공공기관), 문서형태의 구조(RSS)에 사용됨
* df 해당날짜의 항공편별(INC-FUK) 데이터  
  flight: 항공편 (*경로마다 기록포인트 개수가 다름*)    
  time: 시각 (*시간은 UTC기준. 홈페이지는 UTC-5(EST, 미국 동부표준시간대)로 나와서 5만큼 차이남*)    
  longitude: 경도  
  latitude: 위도  
  altitude: 고도  

In [16]:
folder = "/kaggle/input/20251112wed"

df_list = []

# folder 내 모든 파일 순회
for fname in os.listdir(folder):
    fpath = os.path.join(folder, fname)

    print(f"Processing: {fname}...")
    tree = ET.parse(fpath)
    root = tree.getroot()

    ns = {
        "k":"http://www.opengis.net/kml/2.2",
        "gx":"http://www.google.com/kml/ext/2.2"
    } # namespace = 태그의 포인터, 태그이름 같아도 구별필요. 접근용 식별자

    placemarks = root.findall(".//k:Placemark", ns)

    # 경로있는 placemark 탐색
    track_pm = []
    for pm in placemarks:
        track = pm.find("gx:Track", ns)
        if track is not None:
            track_pm = track
            flight = pm.find("k:name", ns).text
            break
    if track_pm is None:
        raise ValueError("경로정보를 찾을 수 없습니다") # 강제 에러발생

    # 경로정보 추출
    whens = [w.text for w in track_pm.findall("k:when", ns)]
    coords = [c.text for c in track_pm.findall("gx:coord", ns)]

    rows = []
    for when, coord in zip(whens, coords):
        when = when.replace("T"," ").replace("Z","") # 2025-11-13T07:10:08Z
        time = pd.to_datetime(when)
        lon, lat, alt = coord.split()
        rows.append([flight, time, float(lon), float(lat), float(alt)])

    df_temp = pd.DataFrame(rows, columns=['flight', 'time', 'longitude', 'latitude', 'altitude'])
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True) # 기존 index를 무시하고 새로 부여. concat에서는 거의 필수

file_name = folder.split("/")[-1]
df.to_csv(f"flight_data_{file_name}.csv", index=False, encoding="utf-8")
print("변환작업 완료!")

Processing: FlightAware_TWB201_RKSI_RJFF_20251112.kml...
Processing: FlightAware_ASV727_RKSI_RJFF_20251112.kml...
Processing: FlightAware_JNA269_RKSI_RJFF_20251112.kml...
Processing: FlightAware_TWB205_RKSI_RJFF_20251112.kml...
Processing: FlightAware_JJA1407_RKSI_RJFF_20251112.kml...
Processing: FlightAware_JNA261_RKSI_RJFF_20251111.kml...
Processing: FlightAware_JJA1405_RKSI_RJFF_20251112.kml...
Processing: FlightAware_KAL787_RKSI_RJFF_20251111.kml...
Processing: FlightAware_ESR647_RKSI_RJFF_20251112.kml...
Processing: FlightAware_ESR649_RKSI_RJFF_20251112.kml...
Processing: FlightAware_ABL156_RKSI_RJFF_20251111.kml...
Processing: FlightAware_JJA1403_RKSI_RJFF_20251112.kml...
Processing: FlightAware_AAR134_RKSI_RJFF_20251112.kml...
Processing: FlightAware_ESR645_RKSI_RJFF_20251111.kml...
Processing: FlightAware_ASV723_RKSI_RJFF_20251111.kml...
Processing: FlightAware_TWB207_RKSI_RJFF_20251112.kml...
Processing: FlightAware_JNA265_RKSI_RJFF_20251112.kml...
Processing: FlightAware_KAL7

In [15]:
df.sample(30)

,flight,time,longitude,latitude,altitude
994,KAL787,2025-11-11 23:51:53,130.04881,33.82594,1585.0
776,JJA1405,2025-11-12 06:29:12,126.40232,37.52403,907.0
224,ASV727,2025-11-12 08:10:21,130.00168,33.74288,2248.0
1249,ESR649,2025-11-12 09:56:42,128.80920,35.06031,8176.0
774,JJA1405,2025-11-12 06:28:40,126.42020,37.50206,533.0
1652,AAR134,2025-11-12 05:00:39,128.57082,35.63951,8839.0
989,KAL787,2025-11-11 23:49:32,129.84478,33.89767,2568.0
145,ASV727,2025-11-12 07:30:34,126.85818,37.35933,5166.0
166,ASV727,2025-11-12 07:41:07,127.83119,36.31284,9456.0
1984,TWB207,2025-11-12 09:20:18,126.61165,37.58432,3269.0


In [ ]:

    tree = ET.parse(fpath)
    root = tree.getroot()

    # KML 네임스페이스
    ns = {"k": "http://www.opengis.net/kml/2.2"}

    placemarks = root.findall(".//k:Placemark", ns)

    rows = []

In [ ]:

# 3) 폴더 내 모든 파일 순회
for fname in os.listdir(folder):
    fpath = os.path.join(folder, fname)

    # 파일 확장자 확인
    ext = fname.split('.')[-1].lower()

    print("Processing:", fname)

    # CSV 파일
    if ext == "csv":
        df = pd.read_csv(fpath)

    # JSON 파일
    elif ext == "json":
        df = pd.read_json(fpath)

    # TXT → CSV처럼 처리 (구분자 공백/탭 자동 감지)
    elif ext == "txt":
        df = pd.read_csv(fpath, sep=None, engine="python")

    # 그 외 확장자는 스킵
    else:
        print("Skip file:", fname)
        continue

    df["source_file"] = fname   # 원본 파일 이름 기록
    df_list.append(df)

# 4) 모든 파일을 하나로 합침
merged_df = pd.concat(df_list, ignore_index=True)

# 5) 최종 CSV 파일로 저장
output_path = "/kaggle/working/merged_output.csv"
merged_df.to_csv(output_path, index=False)

print("통합 CSV 저장 완료:", output_path)


## train데이터를 clustering에 알맞게 preprocessing하는 코드